# Segmenting remote sensing imagery with FastSAM

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/fast_sam.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/fast_sam.ipynb)

FastSAM: https://github.com/CASIA-IVA-Lab/FastSAM

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator. 

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [ ]:
# %pip install segment-geospatial segment-anything-fast

In [ ]:
import leafmap
from samgeo import tms_to_geotiff
from samgeo.fast_sam import SamGeo

## Create an interactive map

In [ ]:
m = leafmap.Map(center=[-22.17615, -51.253043], zoom=18, height="800px")
m.add_basemap("SATELLITE")
m

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [ ]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-51.2565, -22.1777, -51.2512, -22.175]

In [ ]:
image = "Image.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True)

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [ ]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

## Initialize SamGeo class

The initialization of the SamGeo class might take a few minutes. The initialization downloads the model weights and sets up the model for inference.

In [ ]:
from samgeo.fast_sam import SamGeo

sam = SamGeo(model="FastSAM-x.pt")

Set the image.

In [ ]:
sam.set_image("Image.tif")

Segment the image with `everything_prompt`. You can also try `point_prompt`, `box_prompt`, or `text_prompt`.

In [ ]:
sam.everything_prompt(output="mask.tif")

Show the annotated image.

In [ ]:
sam.show_anns("mask.png")

![](https://i.imgur.com/af4bj7O.png)

Convert the segmentation results from GeoTIFF to vector.

In [ ]:
sam.raster_to_vector("mask.tif", "mask.geojson")

Show the segmentation results on the map.

In [ ]:
m.add_raster("mask.tif", opacity=0.5, layer_name="Mask")
m.add_vector("mask.geojson", layer_name="Mask Vector")
m

![](https://i.imgur.com/LvEAMSl.png)